In [1]:
# requirements
from pymongo import MongoClient

import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.neural_network import MLPClassifier

from sklearn.decomposition import PCA

import joblib

import tqdm
import heapq
import json

In [2]:
# MongoDB 연결
client = MongoClient("mongodb://localhost:3999/")

In [3]:
# DB 불러오기
matchup_statistics_db = client["matchup_statistics_db"]  # summoners, indicators, matches

In [4]:
for collection_name in matchup_statistics_db.list_collection_names():
    print(collection_name)
    collection = matchup_statistics_db["indicators"]

    documents = collection.find()
    cnt = 0

    for document in tqdm.tqdm(documents):
        print(document)
        break
        cnt += 1

    print(cnt)

ServerSelectionTimeoutError: localhost:3999: [WinError 10061] 대상 컴퓨터에서 연결을 거부했으므로 연결하지 못했습니다 (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms), Timeout: 30s, Topology Description: <TopologyDescription id: 660caacdab9a99a70f3891e7, topology_type: Unknown, servers: [<ServerDescription ('localhost', 3999) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:3999: [WinError 10061] 대상 컴퓨터에서 연결을 거부했으므로 연결하지 못했습니다 (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms)')>]>

In [31]:
division = {"I": 1, "II": 2, "III": 3, "IV": 4}

diamond_users = []
my_rank = 1

# or (summoner["league"]["tier"] == "DIAMOND" and summoner["league"]["rank"] == "IV")

for summoner in matchup_statistics_db["summoners"].find():
    if (summoner["league"]["tier"] == "DIAMOND" and division[summoner["league"]["rank"]] <= 3):
        diamond_users.append(summoner["_id"])

print(len(diamond_users))
print(diamond_users[0])

3148
t3C2JzNt9v8ypQ3PtL7v4c6VNvn-QdDDnm8-Pcnllyd65xPT


In [32]:
# 학습 데이터 추출

# collection 불러오기
indicators = matchup_statistics_db["indicators"].find()

# 학습 데이터
x = []
y = []

summoners = []

times = 0
all_zero = 0


# 결측치 처리 - 추후에는 평균 값으로 입력
def imputation(datas, indicator, record, record_detail):
    global times

    if datas[indicator][record].get(record_detail):
        return datas[indicator][record].get(record_detail)
    else:
        times += 1
        return 0

# 데이터 추출
for indicator in indicators:
    if indicator["_id"] not in diamond_users:
        continue

    for match_indicator in indicator["matchIndicators"]:
        summoner = {}

        summoner["csDiffer"] = match_indicator["laneIndicator"]["basicWeight"]["csDiffer"]
        summoner["expDiffer"] = match_indicator["laneIndicator"]["basicWeight"]["expDiffer"]
        summoner["turretPlateDestroyDiffer"] = match_indicator["laneIndicator"]["basicWeight"]["turretPlateDestroyDiffer"]
        # summoner["dealDiffer"] = imputation(match_indicator, "laneIndicator", "aggresiveLaneAbilility", "dealDiffer")
        summoner["dealDiffer"] = 

        summoner["turretKillsPerDeaths"] = match_indicator["macroIndicator"]["splitPoint"]["turretKillsPerDeaths"]
        summoner["damageDealtToTurretsPerTeamTotalTowerDamageDone"] = match_indicator["macroIndicator"]["splitPoint"]["damageDealtToTurretsPerTeamTotalTowerDamageDone"]

        summoner["totalTimeCCingOthersPerTotalDamageTaken"] = match_indicator["macroIndicator"]["initiatingPoint"]["totalTimeCCingOthersPerTotalDamageTaken"]
        summoner["totalDamageTakenPerTeamTotalDamageTaken"] = match_indicator["macroIndicator"]["initiatingPoint"]["totalDamageTakenPerTeamTotalDamageTaken"]
        summoner["damageSelfMitigatedPerTotalDamageTaken"] = match_indicator["macroIndicator"]["initiatingPoint"]["damageSelfMitigatedPerTotalDamageTaken"]

        summoner["visionScorePerDeath"] = match_indicator["macroIndicator"]["visionPoint"]["visionScorePerDeath"]
        summoner["totalJungleObjectivePerGameDuration"] = match_indicator["macroIndicator"]["jungleHoldPoint"]["totalJungleObjectivePerGameDuration"]

        summoner["getObjectiveDifferPerGameDuration"] = match_indicator["macroIndicator"]["objectivePoint"]["getObjectiveDifferPerGameDuration"]

        summoner["damagePerMinute"] = match_indicator["macroIndicator"]["totalDealPoint"]["damagePerMinute"]
        summoner["dealPerGold"] = imputation(match_indicator, "macroIndicator", "totalDealPoint", "dealPerGold")
        summoner["teamDamagePercentage"] = imputation(match_indicator, "macroIndicator", "totalDealPoint", "teamDamagePercentage")

        if match_indicator["metadata"]["laneInfo"].get("teamPosition") == "TOP":
            pass

        is_all_zero = True
        idx = 0

        for key, value in summoner.items():
            if value != 0:
                is_all_zero = False
                break 
                
        if is_all_zero is False:

            if match_indicator["metadata"]["isWin"] is True:
                y.append(1)
            else:
                y.append(0)

            summoners.append(summoner)

In [33]:
df = pd.DataFrame(summoners)
df

,csDiffer,expDiffer,turretPlateDestroyDiffer,dealDiffer,turretKillsPerDeaths,damageDealtToTurretsPerTeamTotalTowerDamageDone,totalTimeCCingOthersPerTotalDamageTaken,totalDamageTakenPerTeamTotalDamageTaken,damageSelfMitigatedPerTotalDamageTaken,visionScorePerDeath,totalJungleObjectivePerGameDuration,getObjectiveDifferPerGameDuration,damagePerMinute,dealPerGold,teamDamagePercentage
0,13,760,3.000000,-4025,0,24838,42,16096,40285,100000,0,-300,45300000,121588,0.171219
1,1,789,1.000000,-964,25000,52109,93,18389,41280,325000,0,-263,60600000,138194,0.230803
2,-60,-1797,3.000000,-3732,20000,9717,97,18667,63364,220000,0,-203,29000000,81740,0.081637
3,-6,-547,1.000000,-3761,0,76581,18,15416,46515,150000,0,-169,35800000,98581,0.203790
4,2,-120,-5.000000,815,0,38768,111,21170,35248,100000,0,432,49800000,126718,0.158995
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61769,4,1444,-2.000000,-379,0,18767,53,19323,55894,157142,0,-502,58900000,157740,0.228019
61770,33,2529,-3.000000,8255,50000,18012,185,19534,96992,316666,0,-229,117500000,265628,0.246249
61771,-7,390,0.000000,-1677,0,0,222,18291,82211,1025000,0,-299,39000000,136962,0.143181
61772,-21,713,-0.666667,5227,0,2653,58,33136,107737,322222,857,267,86500000,196161,0.208619


In [34]:
# PCA
df_normalized = (df - df.min()) / (df.max() - df.min())

In [35]:
scaler = StandardScaler()
x = df_normalized.to_numpy()
x_scaled = scaler.fit_transform(x)

In [51]:
print(x[0])

[4.03448276e-01 3.84128847e-01 6.66666667e-01 5.00091767e-01
 0.00000000e+00 0.00000000e+00 2.38235995e-04 5.27419891e-01
 4.80598711e-02 2.11382114e-02 0.00000000e+00 3.21500000e-01
 3.87035872e-02 7.54666457e-02 1.15233087e-01]


In [10]:
parameter_space = {
    'hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 50)],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam'],
    'alpha': [0.0001, 0.05],
    'learning_rate': ['constant','adaptive'],
}

In [37]:
# mlp = MLPClassifier(hidden_layer_sizes=(75, ), max_iter=500, activation='relu', solver='adam', random_state=42, alpha=0.00001, verbose=True)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
mlp = MLPClassifier(max_iter=500)
random_search = RandomizedSearchCV(mlp, param_distributions=parameter_space, n_iter=10, cv=5, verbose=2, random_state=42, n_jobs=-1)
random_search.fit(x_train, y_train)
#mlp.fit(x_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


RandomizedSearchCV(cv=5, estimator=MLPClassifier(max_iter=500), n_jobs=-1,
                   param_distributions={'activation': ['tanh', 'relu'],
                                        'alpha': [0.0001, 0.05],
                                        'hidden_layer_sizes': [(50,), (100,),
                                                               (50, 50),
                                                               (100, 50)],
                                        'learning_rate': ['constant',
                                                          'adaptive'],
                                        'solver': ['sgd', 'adam']},
                   random_state=42, verbose=2)

In [39]:
print("최적의 하이퍼파라미터:", random_search.best_params_)
print("최고 점수:", random_search.best_score_)

최적의 하이퍼파라미터: {'solver': 'adam', 'learning_rate': 'constant', 'hidden_layer_sizes': (100, 50), 'alpha': 0.05, 'activation': 'relu'}
최고 점수: 0.8838299811558897


In [40]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
mlp = MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=500, activation='relu', solver='adam', random_state=42, alpha = 0.05, learning_rate = 'constant', verbose=True)
mlp.fit(x, y)

Iteration 1, loss = 0.45223398
Iteration 2, loss = 0.34403201
Iteration 3, loss = 0.32594876
Iteration 4, loss = 0.31889576
Iteration 5, loss = 0.31383135
Iteration 6, loss = 0.31110588
Iteration 7, loss = 0.30874398
Iteration 8, loss = 0.30603331
Iteration 9, loss = 0.30470982
Iteration 10, loss = 0.30268905
Iteration 11, loss = 0.30168689
Iteration 12, loss = 0.29906680
Iteration 13, loss = 0.29847952
Iteration 14, loss = 0.29875657
Iteration 15, loss = 0.29725730
Iteration 16, loss = 0.29621846
Iteration 17, loss = 0.29418304
Iteration 18, loss = 0.29621153
Iteration 19, loss = 0.29543057
Iteration 20, loss = 0.29295313
Iteration 21, loss = 0.29384495
Iteration 22, loss = 0.29215478
Iteration 23, loss = 0.29095055
Iteration 24, loss = 0.29286381
Iteration 25, loss = 0.29111971
Iteration 26, loss = 0.29087929
Iteration 27, loss = 0.28924422
Iteration 28, loss = 0.28861331
Iteration 29, loss = 0.28970113
Iteration 30, loss = 0.28922416
Iteration 31, loss = 0.28795220
Iteration 32, los

MLPClassifier(alpha=0.05, hidden_layer_sizes=(100, 50), max_iter=500,
              random_state=42, verbose=True)

In [41]:
joblib.dump(mlp, 'models/winning/diamond_1_model.pkl')

['models/winning/diamond_1_model.pkl']

In [15]:
max_heapq = []

probabilities = mlp.predict_proba(x_test)
cnt = 0
total = 0
for i, probability in enumerate(probabilities):
    heapq.heappush(max_heapq, (-probability[1], i))
    total += 1
    if probability[1] > 0.5:
        cnt += 1
    # print(f"{i}번째 승리확률 : {probability[1]}")

print(total)
print(cnt)

for _ in range(5):
    print(heapq.heappop(max_heapq))

19789
10658
(-0.9999997099953462, 9533)
(-0.9999969902798124, 13084)
(-0.9999952915342224, 8674)
(-0.999993736206025, 9599)
(-0.9999918194076449, 16562)
